# Correctness and unambiguity check of algorithms for complementation of SDBA

In [ ]:
%run Evaluation-lib.ipynb
import spot
import os
import pandas as pd
## Imports for progress bars
from ipywidgets import FloatProgress, IntProgress, Latex, HBox
from IPython.display import display

## Unambiguity checking

In [ ]:
def is_unambiguous(path):
    if not is_computed(path):
            return None
    a = !timeout '40s' autfilt --is-unambiguous --stats=%f {path}
    return len(a) == 1 and a[0] == path

The following code runs unambiguity checking and shows the computation progress. The further cells shows the measured values. We check for unambiguity both the original complament and the simplified one.

In [ ]:
def check_unambiguity():
    f = FloatProgress(min=1, max=len(get_cases()))
    overall = HBox([f,Latex('Overal progress')])
    g = FloatProgress(min=1, max=12,bar_style='success')
    label = Latex()
    alg = HBox([g,label])
    mistakes = FloatProgress(min=0, max=0,bar_style='warning')
    mis = HBox([mistakes,Latex('Ratio of unambiguous complements')])
    smistakes = FloatProgress(min=0, max=0,bar_style='warning')
    smis = HBox([smistakes,Latex('Ratio of complements unambiguous after simplifications')])
    res = dict()
    unambiguous,sunambiguous = [],[]
    display(overall,alg,mis,smis)
    for cs in get_cases():
        f.value += 1
        g.max=(len(get_complements(cs)))
        g.value = g.min
        for c in get_complements(cs):
            mistakes.max += 1
            smistakes.max += 1
            label.value = "Algorithm: "+get_algorithm_name(c)
            comp = is_unambiguous(c)
            simp = is_unambiguous(get_simplified_path(c))
            g.value += 1
            if comp:
                mistakes.value += 1
            if simp:
                smistakes.value += 1
            res[(cs,get_algorithm_name(c))]={'unambiguous':comp,'unambiguous_simp':simp}
    return res

In [ ]:
res = check_unambiguity()
ind = pd.MultiIndex.from_tuples(res.keys())
data = pd.DataFrame(res,columns=ind).T
display(data.groupby(level=1).sum())

## Correctness checking
In order to verify whether $C$ is a complement of $A$ we have to check:

1. $L(A_{pr}) = L(A)\cap L(C) = \emptyset$
2. $L(C_{pr}) = \overline{L(A)}\cap \overline{L(C)} = \emptyset$

We already assume here the existence of the products. Path to the products is, for a given path to $C$, returned by `get_products(`$C$`)`. The products can be creating by creating appropriete `Makefile`s using [generate_Makefiles notebook](generate_Makefiles.ipynb). The creation of all necessary automata is a time-demanding task. We recommend to exploit as much paralellism as possible (we used 10 8-core mashines with a shared filesystem). [For IPython Notebook novices: To enable the following two cells, use the `y` key and `r` to disable it again.]

Currently we use a one minute timeout for all emptiness_checks. This value can be changed in definition of function `check_product`.

In [ ]:
def correctness_check(compl):
    def check_product(path,to='1m'):
        if not is_computed(path):
            return None
        # No timeout easily applyable
        #aut = spot.automaton(path)
        #return aut.is_empty()
        res = !timeout {to} autfilt --stats=%f --is-empty {path}
        if len(res) == 1 and res[0] == path:
            return True
        else:
            res2 = !timeout {to} autfilt --stats=%f -v --is-empty {path}
            if len(res) == 1 and res[0] == path:
                return False
        return None
    a_pr,c_pr = get_products(compl)
    return check_product(a_pr),check_product(c_pr)

The following code calls the `correctness_check` function for all automata produced by different tools, and shows progress of the computation. Beware, it make take a very long time.

In [ ]:
tools=['GoalRamsey',
'GoalRank',
"GoalSafraPiterman",
'GoalSlice',
'UltimateBS']

In [ ]:
def check_all():
    f = FloatProgress(min=1, max=len(get_cases()))
    overall = HBox([f,Latex('Overal progress')])
    g = FloatProgress(min=1, max=12,bar_style='success')
    label = Latex()
    alg = HBox([g,label])
    mistakes = FloatProgress(min=0, max=0,bar_style='danger')
    mis = HBox([mistakes,Latex('Ratio of wrong complements')])
    false = []
    display(overall,alg,mis)
    for cs in get_cases():
        f.value += 1
        g.max=(len(get_complements(cs)))
        g.value = g.min
        for c in get_complements(cs):
            if get_algorithm_name(c) in tools:
                mistakes.max += 1
                label.value = "Algorithm: "+get_algorithm_name(c)
                res = correctness_check(c)
                if False in res:
                    mistakes.value += 1
                    false.append(c)
            g.value += 1
    return false

In [ ]:
display(check_all())